In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler

import math

In [ ]:
model = tf.keras.models.load_model('../models/btc_model.h5')

In [ ]:
df = pd.read_csv('../data/raw/BTC-USD.csv')

data = df.filter(['Close'])
dataset = data.values

scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(dataset)

In [ ]:
memory_days = 60

In [ ]:
training_data_len = math.ceil(len(scaled_data) * 0.8)

train_data = scaled_data[:training_data_len, :]
test_data = scaled_data[training_data_len - memory_days:, :]

# Criar conjuntos de dados para treinamento
train_X = []
train_Y = []

for i in range(memory_days, len(train_data)):
    train_X.append(train_data[i-memory_days:i, 0])
    train_Y.append(train_data[i, 0])

# Converter para arrays numpy e redimensionar
train_X = np.array(train_X)
train_Y = np.array(train_Y)

train_X = np.reshape(train_X, (train_X.shape[0], train_X.shape[1], 1))

In [ ]:
test_X = []
test_Y = scaled_data[training_data_len:, :]

for i in range(memory_days, len(test_data)):
    test_X.append(test_data[i-memory_days:i, 0])

test_X = np.array(test_X)
test_X = np.reshape(test_X, (test_X.shape[0], test_X.shape[1], 1))

In [ ]:
# Últimos dados para previsão
last_days_for_seven_days = scaled_data[-memory_days:]  # Pega os últimos 'memory_days' da série
seven_day_predictions = []
prevision_days = 7

# Prever para os próximos 7 dias
for _ in range(prevision_days):
    # Reshape para a entrada do modelo
    last_days_for_seven_days = np.reshape(last_days_for_seven_days, (1, 60, 1))
    
    # Previsão do próximo dia
    next_price = model.predict(last_days_for_seven_days)
    
    # Inverter a escala
    next_price = scaler.inverse_transform(next_price)
    seven_day_predictions.append(next_price[0][0])
    
    # Atualizar last_days com a nova previsão
    next_price_reshaped = next_price.reshape(1, 1, 1)  # Reshape para 3 dimensões
    last_days_for_seven_days = np.append(last_days_for_seven_days[:, 1:, :], next_price_reshaped, axis=1)

# Resultados das previsões
seven_day_predictions = np.array(seven_day_predictions)
print("Previsões para os próximos 7 dias:", seven_day_predictions)

import matplotlib.pyplot as plt

# Supondo que você tenha 'dataset' que contém todos os dados e 'scaled_data' para as previsões
total_data = dataset[:, 0]  # Dados reais
train_size = len(train_data)  # Tamanho dos dados de treino

# Crie um array vazio para armazenar as previsões dos próximos 7 dias
seven_day_full_predictions = np.empty_like(total_data)
seven_day_full_predictions[:] = np.nan  # Preencha com NaN

# Preencher as previsões de 7 dias na posição correta
seven_day_full_predictions[train_size:train_size + prevision_days] = seven_day_predictions

# Configurações do gráfico
plt.figure(figsize=(14, 7))  # Tamanho do gráfico
plt.plot(total_data, label='Dados Reais', color='blue')  # Dados reais
plt.plot(seven_day_full_predictions, label='Previsões para os Próximos 7 Dias', linestyle='--', color='orange')  # Previsões
plt.title('Previsões para os Próximos 7 Dias vs Dados Reais')
plt.xlabel('Dias')
plt.ylabel('Preço')
plt.legend()
plt.show()
